In [1]:
import numpy as np


In [2]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


In [3]:
glove_file ="models/ArModel100.txt"
tmp_file = "models/ArModel100_w2v.txt"

#_ = glove2word2vec(glove_file, tmp_file)

In [4]:

model = KeyedVectors.load_word2vec_format(tmp_file)

In [5]:
model.init_sims(replace=True)

In [6]:
model.most_similar(["управляемый"], topn=25)

[('контролируемый', 0.7408939599990845),
 ('унифицированный', 0.7211440801620483),
 ('мощный', 0.7154970169067383),
 ('инерциальный', 0.7115138173103333),
 ('линейный', 0.707711398601532),
 ('создаваемый', 0.6971192955970764),
 ('радиолокационный', 0.6962533593177795),
 ('безмоторный', 0.6878050565719604),
 ('сконструированный', 0.6872773170471191),
 ('трансляционный', 0.6870273351669312),
 ('планетарный', 0.6854516267776489),
 ('высоконадежный', 0.6850863695144653),
 ('реверсивный', 0.6828446388244629),
 ('усовершенствованный', 0.682282030582428),
 ('инерционный', 0.6791269183158875),
 ('роботизированный', 0.6755259037017822),
 ('гидроакустический', 0.6737356781959534),
 ('сверхмощный', 0.6735060214996338),
 ('коммутационный', 0.6733840107917786),
 ('грузоподъемный', 0.6724337339401245),
 ('пилотажный', 0.6681207418441772),
 ('локатор', 0.6677592992782593),
 ('усилительный', 0.6658490896224976),
 ('баллистический', 0.6646163463592529),
 ('навигационный', 0.6644648909568787)]

In [14]:
def search_neighbours(word, topn=15, min_score=0.4):
    result = set([word])
    if word not in model:
        return result
    neighbs = model.most_similar(word, topn=topn)
    for nei in neighbs:
        #print(nei)
        if nei[1] < min_score:
            break
        result.add(nei[0])
    return result

In [8]:
#import pymorphy2
import pandas as pd

In [15]:
for w in ["сделай", "немного", "тише"]:
    print("Word: {}".format(w))
    s = search_neighbours(w)
    print("synonims: {}".format(s))

Word: сделай
synonims: {'найди', 'скажи', 'принеси', 'запомни', 'рисуй', 'действуй', 'сделай', 'запиши', 'делай', 'нарисуй', 'смотри', 'покажи', 'придумай', 'открой', 'пойди', 'возьми'}
Word: немного
synonims: {'погуще', 'чуточку', 'немного', 'чуть-', 'пресновато', 'подсолила', 'слегка', 'хорошенько', 'хорошенечко', 'чуток', 'чуть-чуть', 'чу-чуть', 'твердоватой', 'немножко', 'перемешаешь', 'поджарился'}
Word: тише
synonims: {'громче', 'спокойнее', 'тише', 'быстрей', 'храпеть', 'бодрее', 'резче', 'спокойней', 'теплей', 'мелодичнее', 'тихо-тихо', 'звонче', 'тихо', 'потише', 'глуше', 'звончей'}


In [10]:
model.save("models/ArModel100_w2v.md")

In [16]:
iot_commands_v01 = pd.read_csv("data/iot_commands_v0.1.csv", sep=";")

print("Total {} rows".format(len(iot_commands_v01)))
variable_text = []
for i, row in iot_commands_v01.iterrows():
    if i % 50 == 0:
        print(i)
    variable_transciptions = [search_neighbours(word, topn=15, min_score=0.4) for word in row.text.split()]
    #print(variable_transciptions)
    #print("Orig: {}".format(row.text))
    out_str = "[{}]".format("] [".join(["|".join(s) for s in variable_transciptions]))
    variable_text.append(out_str)
iot_commands_v01["variable_text"] = variable_text

Total 321 rows
0
50
100
150
200
250
300


In [ ]:
iot_commands_v01.to_csv("data/iot_commands_w2vtext_armodel_v0.1.csv", index_label="id", sep=";")

In [17]:
iot_commands_v01.head(5)

,type,device,condition,location,params,if_schedule,text,variable_text
0,1,1,0,0,0,1,включай стиральную машину в время каждое четно...,[ставь|снимай|врубай|развлекайся|включишь|вклю...
1,1,1,1,0,0,1,когда стиральная машина закончит стирку сообщи...,[почему|идды|уже|отчего|внезапно|ввсё|когда|ес...
2,1,0,0,0,0,0,включи стирку а потом отжим,[отключаешь|отключу|включу|выключу|включаю|сни...
3,3,0,0,0,0,1,сколько раз за месяц я стирал,[посчитайте|уйму|ли|потратишь|склько|скоко|ско...
4,1,0,1,0,0,1,имитируй присутствие людей с число июня до чис...,[имитируй|асока|глумливо|подтолкни|сваливший|ш...


In [37]:
def var_text_str_to_list(var_str):
    var_commands = [v.split("|") for v in var_str.replace("[","").strip()[:-1].split("]")]
    print([ len(s) for s in var_commands])
    print(len(var_commands))
    tokens = []
    #init tokens
    for word in var_commands[0]:
        tokens.append([word])
    #generate seq
    for var_word in var_commands[1:]:
        print(len(tokens))
        new_tokens = []
        for token in tokens:
            for word in var_word:
                new_token = token.copy()
                new_token.append(word)
                new_tokens.append(new_token)
            
        tokens=new_tokens
    return tokens
    
var_text_str_to_list("[a|b][c|d][e|k|l][f]")

[2, 2, 3, 1]
4
2
4
12


[['a', 'c', 'e', 'f'],
 ['a', 'c', 'k', 'f'],
 ['a', 'c', 'l', 'f'],
 ['a', 'd', 'e', 'f'],
 ['a', 'd', 'k', 'f'],
 ['a', 'd', 'l', 'f'],
 ['b', 'c', 'e', 'f'],
 ['b', 'c', 'k', 'f'],
 ['b', 'c', 'l', 'f'],
 ['b', 'd', 'e', 'f'],
 ['b', 'd', 'k', 'f'],
 ['b', 'd', 'l', 'f']]

In [38]:
def var_df_to_commands(df):
    commands = {"parent_id":[], "command":[]}
    for i, row in df.iterrows():
        print(i)
        com = var_text_str_to_list(row.variable_text)
        print(com)
        parent_id = [i] * len(com)
        commands["parent_id"].extend(parent_id)
        commands["command"].extend(com)
    return pd.DataFrame(commands)


In [ ]:
var_df_to_commands(iot_commands_v01.head(1))

0
[16, 16, 16, 16, 16, 16, 16, 16, 16]
9
16
256
4096
65536
1048576
